In [1]:
import pandas as pd
import datetime
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import binarize
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score,f1_score ,roc_auc_score, explained_variance_score, mean_squared_error, r2_score, plot_roc_curve

print(torch.__version__)
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)

1.9.0+cu111
Populating the interactive namespace from numpy and matplotlib


# Pandas Datareader Test

In [2]:
total = pd.read_csv('./result/total.csv')
print(len(total))

4804


In [3]:
print(len(total))
temp = total[total['is_infection']==1]
total = pd.concat([total, temp])
total = pd.concat([total, temp])
e = len(total[total['is_infection']==1])
print(e*4/len(total))

4804
0.46331156358048403


In [4]:
total.columns
# print(type(total))
print(type(total['Unnamed: 0']))
total['Unnamed: 0'] = pd.to_datetime(total['Unnamed: 0'])
total.columns

<class 'pandas.core.series.Series'>


Index(['Unnamed: 0', 'subject_id', 'stay_id', 'compiled_hr', 'endtime',
       'pao2ratio_vent', 'rate_dopamine', 'rate_epinephrine',
       'rate_nonepinephrine', 'meabp_min', 'heartrate_max', 'temperature_max',
       'gcs_min', 'bilirubin_max', 'creatineine_max', 'paltelet_min',
       'respiration', 'coagulation', 'liver', 'cns', 'renal',
       'respiration_24hours', 'coagulation_24hours', 'liver_24hours',
       'cns_24hours', 'renal_24hours', 'sofa_24hours', 'infection_time',
       'is_infection', 'infection_hour'],
      dtype='object')

In [5]:
total.head(10)

,Unnamed: 0,subject_id,stay_id,compiled_hr,endtime,pao2ratio_vent,rate_dopamine,rate_epinephrine,rate_nonepinephrine,meabp_min,...,renal,respiration_24hours,coagulation_24hours,liver_24hours,cns_24hours,renal_24hours,sofa_24hours,infection_time,is_infection,infection_hour
0,2112-02-16 19:00:00,14034916,39521325,0,2112-02-17 03:00:00,1.0,0.0,0.0,0.0,0.000000,...,0.0,0.5,0.0,0.0,0.0,0.0,0.50,0,0.0,0.0
1,2112-02-17 03:00:00,14034916,39521325,1,2112-02-17 11:00:00,0.0,0.0,0.0,0.0,0.147887,...,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0,0.0,0.0
2,2112-02-17 11:00:00,14034916,39521325,2,2112-02-17 19:00:00,0.0,0.0,0.0,0.0,0.866197,...,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0,0.0,0.0
3,2112-02-17 19:00:00,14034916,39521325,3,2112-02-18 03:00:00,0.0,0.0,0.0,0.0,0.133803,...,0.0,1.0,0.0,0.0,0.0,0.0,1.00,0,0.0,0.0
4,2131-10-11 14:00:00,12588462,32189053,0,2131-10-11 22:00:00,0.0,0.0,0.0,0.0,0.008621,...,0.0,0.0,1.0,0.0,1.0,0.0,0.50,0,0.0,0.0
5,2131-10-11 22:00:00,12588462,32189053,1,2131-10-12 06:00:00,0.0,0.0,0.0,0.0,0.212644,...,0.0,0.0,1.0,0.0,1.0,0.0,0.50,0,0.0,0.0
6,2131-10-12 06:00:00,12588462,32189053,2,2131-10-12 14:00:00,0.0,0.0,0.0,0.0,0.540230,...,0.0,1.0,1.0,0.0,1.0,0.0,1.00,0,0.0,0.0
7,2131-10-12 14:00:00,12588462,32189053,3,2131-10-12 22:00:00,0.0,0.0,0.0,0.0,0.413793,...,0.0,1.0,1.0,0.0,0.0,0.0,0.75,0,0.0,0.0
8,2128-10-15 03:00:00,17188632,31948656,0,2128-10-15 11:00:00,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,0.0,0.0
9,2128-10-15 11:00:00,17188632,31948656,1,2128-10-15 19:00:00,0.0,0.0,0.0,0.0,0.429630,...,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0,0.0,0.0


In [6]:
total=total.set_index('Unnamed: 0')
# total=total.sort_values(by=['Unnamed: 0'])
total = total.drop(['subject_id', 'stay_id', 'endtime', 'infection_time', 'infection_hour', 'compiled_hr'], axis=1)
#total = total.astype('float')

In [7]:
total.columns
print(total.index)

DatetimeIndex(['2112-02-16 19:00:00', '2112-02-17 03:00:00',
               '2112-02-17 11:00:00', '2112-02-17 19:00:00',
               '2131-10-11 14:00:00', '2131-10-11 22:00:00',
               '2131-10-12 06:00:00', '2131-10-12 14:00:00',
               '2128-10-15 03:00:00', '2128-10-15 11:00:00',
               ...
               '2143-11-30 13:00:00', '2184-06-17 05:00:00',
               '2156-12-12 08:00:00', '2129-07-11 19:00:00',
               '2118-01-14 05:00:00', '2180-02-21 07:00:00',
               '2186-06-09 17:00:00', '2165-09-25 08:00:00',
               '2181-01-10 05:00:00', '2114-10-10 07:00:00'],
              dtype='datetime64[ns]', name='Unnamed: 0', length=5206, freq=None)


In [8]:
print(total.values)

[[1.  0.  0.  ... 0.  0.5 0. ]
 [0.  0.  0.  ... 0.  1.  0. ]
 [0.  0.  0.  ... 0.  1.  0. ]
 ...
 [0.  0.  0.  ... 0.  1.  1. ]
 [1.  0.  0.  ... 0.  0.1 1. ]
 [0.  0.  0.  ... 0.  0.  1. ]]


In [9]:
print(total.columns)

Index(['pao2ratio_vent', 'rate_dopamine', 'rate_epinephrine',
       'rate_nonepinephrine', 'meabp_min', 'heartrate_max', 'temperature_max',
       'gcs_min', 'bilirubin_max', 'creatineine_max', 'paltelet_min',
       'respiration', 'coagulation', 'liver', 'cns', 'renal',
       'respiration_24hours', 'coagulation_24hours', 'liver_24hours',
       'cns_24hours', 'renal_24hours', 'sofa_24hours', 'is_infection'],
      dtype='object')


# Data Preparation

In [10]:
class SepsisDataset(torch.utils.data.Dataset):
    def __init__(self, df_data, seq_len=4):
        self.seq_len = seq_len
        self.X = df_data.loc[:,df_data.columns!='is_infection'].values
        self.y = df_data.loc[:, 'is_infection'].values
        
    def __len__(self):
        return len(self.X)//self.seq_len
    
    def __getitem__(self, idx):
        idx += self.seq_len # 몇 시간 단위로 인덱스 올리는 거
        X = self.X[idx-self.seq_len:idx-2]
        y = self.y[idx-1]
        return X, y 

In [11]:
train_dataset = SepsisDataset(total, seq_len=4)
trainloader = DataLoader(train_dataset,  # dataset을 input으로
                             batch_size=1000,
                             shuffle=True, drop_last=True)

for i, (X, y) in enumerate(trainloader) :
    print(X.size() , y.size())

torch.Size([1000, 2, 22]) torch.Size([1000])


# Model Define

In [12]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, num_layers, batch_size, dropout, use_bn):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = 1
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        # 나중에 network(node 개수 바꿔 주는 옵션도 생각하기)
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        layers.append(nn.Sigmoid())
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
      # Model Capacity 따라 y.sahpe이 달라짐
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1)) # 맨 마지막 h_t 갖고 예측 할거니까 -1
#         t = Variable(torch.Tensor([0.5]))  # threshold
#         out = (y_pred > t).float() * 1
        return y_pred

In [13]:
def metric(y_pred, y_true):  
#     auc = roc_auc_score(y_true, y_pred) 
    
    perc_y_pred = y_pred.cpu().detach().numpy()
    perc_y_true = y_true.cpu().detach().numpy()

#     try:
#         auc = roc_auc_score(perc_y_true, perc_y_pred)
#     except ValueError:
#         print('Error')
#         auc = 0.99

#     print('@@@@@@@@@@@@@')
#     print(perc_y_pred)
#     print(perc_y_true)
    perc_y_pred = np.where(perc_y_pred > 0.5, 1, 0)
    auc = accuracy_score(perc_y_true, perc_y_pred, normalize=True)
#     print(auc)
    return auc

# Train, Validate, Test 

In [14]:
def train(model, partition, optimizer, loss_fn, args):
  # batch 단위로 갖고 오는 것
  # 무작위로 시작 record 갖고 옴
  # 갖고 올때 batch size만큼 갖고 옴
  # permutation, not random
    trainloader = DataLoader(partition['train'],  # dataset을 input으로
                             batch_size=args.batch_size,
                             shuffle=True, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    train_acc = 0.0
    train_loss = 0.0
    for i, (X, y) in enumerate(trainloader):

        # X : [10, n, 6] [n, 10, 6] 현재 타임스템프 안에
        # Y : [10, m, 1] 
        X = X.transpose(0, 1).float().to(args.device)
        y_true = y.float().to(args.device) 

        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += metric(y_pred, y_true)

    train_loss = train_loss / len(trainloader)
    train_acc = train_acc / len(trainloader)
    
    return model, train_loss, train_acc

In [15]:
def validate(model, partition, loss_fn, args):
    valloader = DataLoader(partition['val'], 
                           batch_size=args.batch_size, 
                           shuffle=False, drop_last=True)
    model.eval()

    val_acc = 0.0
    val_loss = 0.0
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y.float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()
            val_acc += metric(y_pred, y_true)

    val_loss = val_loss / len(valloader)
    val_acc = val_acc / len(valloader)
    return val_loss, val_acc

In [16]:
def test(model, partition, args):
    testloader = DataLoader(partition['test'], 
                           batch_size=args.batch_size, 
                           shuffle=False, drop_last=True)
    model.eval()

    test_acc = 0.0
    cnt = 0
    with torch.no_grad():
        for i, (X, y) in enumerate(testloader):
            cnt += 1
#             print(cnt)
            X = X.transpose(0, 1).float().to(args.device)
            y_true = y.float().to(args.device)
            model.hidden = [hidden.to(args.device) for hidden in model.init_hidden()]

            y_pred = model(X)
            if cnt == 1:
                y_true_all = torch.tensor(y_true)
                y_pred_all = torch.tensor(y_pred)
            else:
                y_true_all = torch.cat([y_true_all, y_true], dim=0)
                y_pred_all = torch.cat([y_pred_all, y_pred], dim=0)
                
            test_acc += metric(y_pred, y_true)
        
    test_acc = test_acc / len(testloader)
#     print(len(y_pred_all), len(y_true_all))
    
    return y_pred, test_acc, y_true_all, y_pred_all

In [17]:
def experiment(partition, args):

    model = LSTM(args.input_dim, args.hid_dim, args.n_layers, args.batch_size, args.dropout, args.use_bn)
    model.to(args.device)

    # loss_fn = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
    loss_fn = nn.BCELoss()
    if args.optim == 'SGD':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    # ===================================== #
    
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        model, train_loss, train_acc = train(model, partition, optimizer, loss_fn, args)
        val_loss, val_acc = validate(model, partition, loss_fn, args)
        te = time.time()
        
        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        # ============================ #
        
        if epoch % 10 == 0:
            print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    y_pred, test_acc, y_true, y_pred_all = test(model, partition, args)   
    print('len of temp:', len(temp))
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc
#     result['pred'] = y_pred.cpu().detach().numpy()
    return vars(args), result, y_true, y_pred_all

# Manage Experiment

In [18]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):
    exp_name = setting['exp_name']
    del setting['epoch']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

In [19]:

def plot_acc(var1, var2, df):

    fig, ax = plt.subplots(1, 3)
    fig.set_size_inches(15, 6)
    sns.set_style("darkgrid", {"axes.facecolor": ".9"})

    sns.barplot(x=var1, y='train_acc', hue=var2, data=df, ax=ax[0])
    sns.barplot(x=var1, y='val_acc', hue=var2, data=df, ax=ax[1])
    sns.barplot(x=var1, y='test_acc', hue=var2, data=df, ax=ax[2])
    
    ax[0].set_title('Train Accuracy')
    ax[1].set_title('Validation Accuracy')
    ax[2].set_title('Test Accuracy')

    
def plot_loss_variation(var1, var2, df, **kwargs):

    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_losses = list(row.train_losses)[0]
            val_losses = list(row.val_losses)[0]

            for epoch, train_loss in enumerate(train_losses):
                list_data.append({'type':'train', 'loss':train_loss, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_loss in enumerate(val_losses):
                list_data.append({'type':'val', 'loss':val_loss, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'loss', marker='.')
    g.add_legend()
    g.fig.suptitle('Train loss vs Val loss')
    plt.subplots_adjust(top=0.89) # 만약 Title이 그래프랑 겹친다면 top 값을 조정해주면 됩니다! 함수 인자로 받으면 그래프마다 조절할 수 있겠죠?


def plot_acc_variation(var1, var2, df, **kwargs):
    list_v1 = df[var1].unique()
    list_v2 = df[var2].unique()
    list_data = []

    for value1 in list_v1:
        for value2 in list_v2:
            row = df.loc[df[var1]==value1]
            row = row.loc[df[var2]==value2]

            train_accs = list(row.train_accs)[0]
            val_accs = list(row.val_accs)[0]
            test_acc = list(row.test_acc)[0]

            for epoch, train_acc in enumerate(train_accs):
                list_data.append({'type':'train', 'Acc':train_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})
            for epoch, val_acc in enumerate(val_accs):
                list_data.append({'type':'val', 'Acc':val_acc, 'test_acc':test_acc, 'epoch':epoch, var1:value1, var2:value2})

    df = pd.DataFrame(list_data)
    g = sns.FacetGrid(df, row=var2, col=var1, hue='type', **kwargs)
    g = g.map(plt.plot, 'epoch', 'Acc', marker='.')

    def show_acc(x, y, metric, **kwargs):
        plt.scatter(x, y, alpha=0.3, s=1)
        metric = "Test Acc: {:1.3f}".format(list(metric.values)[0])
        plt.text(0.05, 0.95, metric,  horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes, bbox=dict(facecolor='yellow', alpha=0.5, boxstyle="round,pad=0.1"))
    g = g.map(show_acc, 'epoch', 'Acc', 'test_acc')

    g.add_legend()
    g.fig.suptitle('Train Accuracy vs Val Accuracy')
    plt.subplots_adjust(top=0.89)

In [20]:
train_val_dataset = total[:int((len(total) / 4)*.8)*4]
test_dataset = total[int((len(total) / 4)*.8)*4:]
train_dataset = train_val_dataset[:int((len(train_val_dataset) / 4)*.8)*4]
valid_dataset = train_val_dataset[int((len(train_val_dataset) / 4)*.8)*4:]

trainset = SepsisDataset(train_dataset, seq_len=4)
valset = SepsisDataset(valid_dataset, seq_len=4)
testset = SepsisDataset(test_dataset, seq_len=4)

print(len(test_dataset)/4)

partition = {'train': trainset, 'val':valset, 'test':testset}

260.5


In [ ]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_lr"
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'
args.batch_size = 64

# ====== Model Capacity ===== #
args.input_dim = 22
args.hid_dim = 50
args.n_layers = 2

# ====== Regularization ======= #
args.l2 = 0.00001
args.dropout = 0.0
args.use_bn = True

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0001
args.epoch = 50


# ====== Experiment Variable ====== #
name_var1 = 'lr'
name_var2 = 'n_layers'
list_var1 = [0.001, 0.0001, 0.00001]
list_var2 = [1,2,3]

for var1 in list_var1:
    for var2 in list_var2:
        setattr(args, name_var1, var1)
        setattr(args, name_var2, var2)
        print(args)
                
        setting, result, y_true, y_pred_all = experiment(partition, deepcopy(args))
        save_exp_result(setting, result)

Namespace(batch_size=64, device='cuda', dropout=0.0, epoch=50, exp_name='exp1_lr', hid_dim=50, input_dim=22, l2=1e-05, lr=0.001, n_layers=1, optim='Adam', use_bn=True)
Epoch 0, Acc(train/val): 0.79/0.96, Loss(train/val) 0.62128/0.61812. Took 0.08 sec
Epoch 10, Acc(train/val): 0.96/0.96, Loss(train/val) 0.15288/0.15770. Took 0.05 sec
Epoch 20, Acc(train/val): 0.96/0.96, Loss(train/val) 0.11822/0.15890. Took 0.07 sec
Epoch 30, Acc(train/val): 0.97/0.97, Loss(train/val) 0.09397/0.17362. Took 0.08 sec
Epoch 40, Acc(train/val): 0.97/0.95, Loss(train/val) 0.07898/0.20987. Took 0.06 sec


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


len of temp: 201
Namespace(batch_size=64, device='cuda', dropout=0.0, epoch=50, exp_name='exp1_lr', hid_dim=50, input_dim=22, l2=1e-05, lr=0.001, n_layers=2, optim='Adam', use_bn=True)
Epoch 0, Acc(train/val): 0.86/0.22, Loss(train/val) 0.56533/0.69596. Took 0.11 sec
Epoch 10, Acc(train/val): 0.96/0.96, Loss(train/val) 0.13799/0.17580. Took 0.11 sec
Epoch 20, Acc(train/val): 0.96/0.94, Loss(train/val) 0.10388/0.20437. Took 0.09 sec
Epoch 30, Acc(train/val): 0.97/0.92, Loss(train/val) 0.08066/0.22249. Took 0.07 sec
Epoch 40, Acc(train/val): 0.97/0.90, Loss(train/val) 0.07145/0.27980. Took 0.08 sec


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


len of temp: 201
Namespace(batch_size=64, device='cuda', dropout=0.0, epoch=50, exp_name='exp1_lr', hid_dim=50, input_dim=22, l2=1e-05, lr=0.001, n_layers=3, optim='Adam', use_bn=True)
Epoch 0, Acc(train/val): 0.96/0.96, Loss(train/val) 0.50656/0.59160. Took 0.09 sec
Epoch 10, Acc(train/val): 0.96/0.96, Loss(train/val) 0.13366/0.16047. Took 0.11 sec
Epoch 20, Acc(train/val): 0.96/0.95, Loss(train/val) 0.10757/0.17954. Took 0.11 sec
Epoch 30, Acc(train/val): 0.97/0.94, Loss(train/val) 0.09272/0.20273. Took 0.07 sec
Epoch 40, Acc(train/val): 0.97/0.92, Loss(train/val) 0.08136/0.24205. Took 0.09 sec


...

len of temp: 201
Namespace(batch_size=64, device='cuda', dropout=0.0, epoch=50, exp_name='exp1_lr', hid_dim=50, input_dim=22, l2=1e-05, lr=1e-05, n_layers=1, optim='Adam', use_bn=True)
Epoch 0, Acc(train/val): 0.28/0.70, Loss(train/val) 0.75813/0.68736. Took 0.07 sec
Epoch 10, Acc(train/val): 0.33/0.24, Loss(train/val) 0.73963/0.76276. Took 0.06 sec
Epoch 20, Acc(train/val): 0.37/0.30, Loss(train/val) 0.72080/0.74297. Took 0.06 sec
Epoch 30, Acc(train/val): 0.43/0.32, Loss(train/val) 0.70352/0.72576. Took 0.06 sec
Epoch 40, Acc(train/val): 0.49/0.35, Loss(train/val) 0.68750/0.70709. Took 0.06 sec


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


len of temp: 201
Namespace(batch_size=64, device='cuda', dropout=0.0, epoch=50, exp_name='exp1_lr', hid_dim=50, input_dim=22, l2=1e-05, lr=1e-05, n_layers=2, optim='Adam', use_bn=True)
Epoch 0, Acc(train/val): 0.07/0.04, Loss(train/val) 0.84613/0.78050. Took 0.06 sec
Epoch 10, Acc(train/val): 0.11/0.06, Loss(train/val) 0.81500/0.83266. Took 0.06 sec
Epoch 20, Acc(train/val): 0.15/0.10, Loss(train/val) 0.78541/0.80303. Took 0.06 sec
Epoch 30, Acc(train/val): 0.23/0.16, Loss(train/val) 0.75879/0.77511. Took 0.06 sec
Epoch 40, Acc(train/val): 0.30/0.26, Loss(train/val) 0.73350/0.74968. Took 0.06 sec


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


len of temp: 201
Namespace(batch_size=64, device='cuda', dropout=0.0, epoch=50, exp_name='exp1_lr', hid_dim=50, input_dim=22, l2=1e-05, lr=1e-05, n_layers=3, optim='Adam', use_bn=True)
Epoch 0, Acc(train/val): 0.81/0.96, Loss(train/val) 0.65807/0.67187. Took 0.07 sec
Epoch 10, Acc(train/val): 0.91/0.89, Loss(train/val) 0.64098/0.64474. Took 0.07 sec
Epoch 20, Acc(train/val): 0.94/0.95, Loss(train/val) 0.62292/0.62512. Took 0.08 sec


In [ ]:
var1 = 'lr'
var2 = 'n_layers'
df = load_exp_result('exp1')

plot_acc(var1, var2, df)
plot_loss_variation(var1, var2, df, sharey=False) #sharey를 True로 하면 모둔 subplot의 y축의 스케일이 같아집니다.
plot_acc_variation(var1, var2, df, margin_titles=True, sharey=True)

In [ ]:
y_pred_all = y_pred_all.cpu().detach().numpy()
y_true = y_true.cpu().detach().numpy()
y_pred_all = np.where(y_pred_all > 0.5, 1, 0)

print(len(y_pred_all))
print(len(y_true))

auc = accuracy_score(y_true, y_pred_all, normalize=True)

In [ ]:
auc


In [ ]:
from sklearn.metrics import classification_report

t = classification_report(y_true, y_pred_all, target_names=['0', '1'])

In [ ]:
print(type(t))

In [ ]:
y_pred_all.sum()


In [ ]:
y_true.sum()

In [ ]:
print(t)

In [ ]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    tt = confusion[0,0]
    tf = confusion[0,1]
    ft = confusion[1,0]
    ff = confusion[1,1]
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='weighted') # None, 'micro', 'macro', 'weighted'
    recall = recall_score(y_test, y_pred,average='weighted')
    F1 = f1_score(y_test, y_pred,average='weighted')
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))    
    print('양성예측율: {:.4f}'.format(tt/(tt+tf)))
    print('음성예측율: {:.4f}'.format(ff/(ft+ff)))    
    print('F1: {:.4f}'.format(F1))

In [ ]:
get_clf_eval(y_true, y_pred_all)